In [19]:
# 패키지 설치
import pandas as pd
#warning 메시지 표시 안함
import warnings
from konlpy.tag import Okt

warnings.filterwarnings(action = 'ignore')

# 형태소 분석에 사용할 konlpy 패키지의 Okt 클래스를 임포트하고 okt객체를 생성
okt = Okt()

# Train 데이터 불러오기
train_df = pd.read_excel('result.xlsx')

# 데이터 확인
print(train_df.head())

                                      text
0                                      NaN
1  여기서 나라 망했네 드립치는 애들은 뭐다? 망한각 잘보는 애들은 뭐다?
2                             이세계아이돌 최고!!!
3          진짜 진입장벽 없이 입덕부정기 없이 들어온 이퐈리임다ㅋㅋ
4                              양지러 올라와 버렸다


In [24]:
# 댓글이 있는 항목만 담기(빈 댓글 삭제)
# text 컬럼이 non-null인 샘플만 train_df에 다시 저장
train_df = train_df[train_df['text'].notnull()]

# 수정된 train_df의 정보를 다시 확인
print(train_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 1164 entries, 1 to 1164
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1164 non-null   object
dtypes: object(1)
memory usage: 18.2+ KB
None


In [35]:
# 한글 외 문자 제거(옵션)
import re # 정규식을 사용하기 위해 re 모듈을 임포트
# ‘ㄱ ~‘힣’까지의 문자를 제외한 나머지는 공백으로 치환, 영문: a-z| A-Z
train_df['text'] = train_df['text'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
print(train_df.head())

# Train용 데이터셋의 정보를 재확인
print(train_df.info())
text = train_df['text']
score = train_df['score']

                                                text
1            여기서 나라 망했네 드립치는 애들은 뭐다  망한각 잘보는 애들은 뭐다 
2                                         이세계아이돌 최고 
3                    진짜 진입장벽 없이 입덕부정기 없이 들어온 이퐈리임다ㅋㅋ
4                                        양지러 올라와 버렸다
5  진심으로 늘 같은 곳만을 바라봐주는 이세계 아이돌 사랑합니다  앞으로도 스브스뉴스같...
<class 'pandas.core.frame.DataFrame'>
Index: 1164 entries, 1 to 1164
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1164 non-null   object
dtypes: object(1)
memory usage: 18.2+ KB
None


In [37]:
# Train용 데이터셋과 Test용 데이터 셋 분리
# 1. 예측력을 높이기 위해 수집된 데이터를 학습용과 테스트 용으로 분리하여 진행
# 2. 보통 20~30%를 테스트용으로 분리해 두고 테스트

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, _= train_test_split(text, None, test_size=0.2, random_state=0)
print(len(train_x), len(test_x))

TypeError: 'NoneType' object is not subscriptable

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
print(tfv_train_x)

  (0, 1427)	0.18177725754932203
  (0, 1341)	0.2036599898876243
  (0, 1340)	0.11713137041432584
  (0, 1255)	0.17172363841092905
  (0, 1229)	0.18599285822622683
  (0, 941)	0.10267752954992251
  (0, 919)	0.09141726254041796
  (0, 907)	0.22140745823685665
  (0, 881)	0.2968751181282122
  (0, 767)	0.16411012588792454
  (0, 715)	0.2036599898876243
  (0, 696)	0.15130954805669952
  (0, 694)	0.18599285822622683
  (0, 434)	0.17172363841092905
  (0, 410)	0.2036599898876243
  (0, 407)	0.19661532355502456
  (0, 404)	0.1252112150414924
  (0, 301)	0.18374303311255305
  (0, 237)	0.19661532355502456
  (0, 153)	0.2036599898876243
  (0, 148)	0.18177725754932203
  (0, 130)	0.12942681571839726
  (0, 84)	0.4073199797752486
  (0, 83)	0.3328994453562492
  (2, 1340)	0.20776296269691086
  :	:
  (926, 143)	0.33405307264588313
  (927, 1199)	0.35467103385443205
  (927, 950)	0.10577005737078558
  (927, 942)	0.35467103385443205
  (927, 941)	0.1788114866311448
  (927, 919)	0.15920188857537754
  (927, 256)	0.3323790059

In [33]:
from sklearn.linear_model import LogisticRegression # 이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=0)
params = {'C': [15, 18, 19, 20, 22]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_x)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

Fitting 3 folds for each of 5 candidates, totalling 15 fits
{'C': 19} 0.013964664799640425
